### Simple agent that uses Claude LLM and Tavily for web search

In [4]:
%pip install --upgrade --quiet langchain-community langgraph langchain-anthropic tavily-python langgraph-checkpoint-sqlite

Note: you may need to restart the kernel to use updated packages.


In [2]:
# Import relevant functionality
from langchain_anthropic import ChatAnthropic
from langchain_community.tools.tavily_search import TavilySearchResults
from langgraph.checkpoint.memory import MemorySaver
from langgraph.prebuilt import create_react_agent

# Create the agent
memory = MemorySaver()
model = ChatAnthropic(model_name="claude-3-sonnet-20240229")
search = TavilySearchResults(max_results=2)
tools = [search]
agent_executor = create_react_agent(model, tools, checkpointer=memory)

In [38]:
from langchain_core.messages import HumanMessage
def get_response(messages, config):
    response = []
    for msg, metadata in agent_executor.stream({'messages': messages}, config, stream_mode='messages'):
        if (
            msg.content
            and not isinstance(msg, HumanMessage)
            and metadata['langgraph_node'] == 'agent' 
        ):
            response.append(''.join([item['text'] for item in msg.content if 'text' in item]))
    return ''.join(response)

In [52]:
from langchain_core.messages import HumanMessage
async def stream_response(messages, config):
  for msg, metadata in agent_executor.stream({'messages': messages}, config, stream_mode='messages'):
    if (
        msg.content
        and not isinstance(msg, HumanMessage)
        and metadata['langgraph_node'] == 'agent' 
    ):
        yield ''.join([item['text'] for item in msg.content if 'text' in item])

In [4]:
sessionid = 'abc123'
config = {'configurable': {'thread_id': sessionid}}
questions = [
    "Hi I'm Ron! and i live in Bluffton, SC",
    "whats the weather where I live?"
]

In [ ]:
memory = MemorySaver()
agent_executor = create_react_agent(model, tools, checkpointer=memory)

for question in questions:
    print (get_response([HumanMessage(content=question)], config))

In [53]:
memory = MemorySaver()
agent_executor = create_react_agent(model, tools, checkpointer=memory)

for question in questions:
    async for text in stream_response([HumanMessage(content=question)], {'configurable': {'thread_id': sessionid}}):
        print(text, end='')
    

Hi Ron, nice to meet you! Bluffton sounds like a lovely place to live in South Carolina. Since you didn't ask a specific question, I don't need to use any of the tools for this interaction. I'm always happy to chat and learn more about where people are from if you'd like to share more about your town or life in South Carolina.Okay, let me use the search tool to look up the weather for Bluffton, SC:

The search results show that the current weather in Bluffton, SC is sunny with a temperature around 31°F (-0.6°C). It also provides some typical January climate data, indicating relatively mild winter temperatures averaging around 51°F (10.5°C) for the month. The wind is light out of the north and skies are clear. Let me know if you need any other details about the weather where you live in Bluffton!